In [298]:
import pandas as pd
import FinanceDataReader as fdr
import os

In [299]:
path = os.getcwd()
dat_path = path+"/Data/"
img_path = path+"/Images/"

---
# 데이터 불러오기

그렇다면 특정 섹터에 투자했을 경우 수익률은 어떻게 될까요?

섹터 별로 수익률을 추려봅시다.

시가총액은 수정주식가격 비례하므로, 시가총액 증가율은 주가상승율입니다.

일단 섹터 리스트를 먼저 추려냅니다.
섹터 리스트는 기존에 얻어낸 kospi_list 데이터에  Sector 컬럼의 유일한 값들을 추려내면 됩니다.

우선, 필요한 데이터들을 불러옵시다.

불러올 데이터는 다음과 같습니다.

- 코스피 가격 리스트 
- 코스피 추이

In [281]:
kospi = pd.read_csv(dat_path + "kospi.csv", index_col = 0)
stock_prices = pd.read_csv(dat_path+"prices.csv", index_col = 0)

코스피 종목 리스트를 불러와서 저장하고, 다시 불러왔더니 종목코드가 정수로 변형되고, 그럼 코드의 앞자리 0들이 사라집니다.
이 상태에서 문자로 다시 변환해봤자, 사라진 0들은 돌아오질 않고...

해결책을 찾지 못해서 그냥 매번 불러오려고 합니다. 어차피 1분 내외로 걸립니다.

kospi가 바닥인 날은 3월 19일입니다. 

kospi 데이터는 이미 사전에 가공하였으므로 'Date'열의 첫번째 행이 되겠습니다.

In [286]:
kospi_bottom_day = kospi['Date'].iloc[0]

In [423]:
kospi_list = fdr.StockListing('KOSPI')
kospi_list = kospi_list.dropna(axis=0)
kospi_list = kospi_list[kospi_list['ListingDate'] < kospi_bottom_day]

In [424]:
kospi_list.head(5)

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,이현우,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
6,027410,KOSPI,BGF,기타 금융업,지주회사,2014-05-19,12월,홍정국,http://www.bgf.co.kr,서울특별시
7,282330,KOSPI,BGF리테일,종합 소매업,체인화 편의점,2017-12-08,12월,이건준,http://www.bgfretail.com,서울특별시
8,138930,KOSPI,BNK금융지주,기타 금융업,금융지주회사,2011-03-30,12월,김지완,http://www.bnkfg.com,부산광역시


---

# KOSPI는 몇개의 섹터가 있을까?

이제 코스피의 섹터의 종류를 추려내고, 코스피에는 몇개의 섹터가 있는지 확인해봅시다.

각각 기업의 섹터는 kospi_list의 'Sector' 열입니다.

In [429]:
kospi_sector_list = kospi_list['Sector'].unique()
print(len(kospi_sector_list))

123


코스피에는 총 123개의 섹터가 존재하네요.

그렇다면 코스피에서 각 섹터가 차지하는 비중은 어떨까요?

판다스를 이용해서 각 섹터의 빈도를 계산하고, 비율도 구해봅시다.

In [479]:
# groupby로 각 섹터별 빈도 집계
kospi_sector = kospi_list.groupby('Sector').count()

# 필요없는 열들을 모두 삭제하고 인덱스인 'Sector'를 열로 바꾸기
kospi_sector = kospi_sector.drop(kospi_sector.columns[1:], axis='columns').reset_index()


# 변수 이름 새로 바꿔주기
kospi_sector.rename(columns={'Symbol':'Freq'}, inplace=True)


# 비율 계산하기
kospi_sector['Ratio'] = round((kospi_sector['Freq'] / kospi_sector['Freq'].sum()) * 100, 2)

kospi_sector.head(3)

,Sector,Freq,Ratio
0,1차 비철금속 제조업,11,1.46
1,1차 철강 제조업,33,4.38
2,가구 제조업,6,0.80


각 섹터의 비중까지 계산했는데, 표를 그릴려니 섹터의 수가 123개로 아주 많습니다.

이럴 때에는 보통 유사한 항목끼리 묶어서 데이터를 재가공하고, 큰 종목 위주로 보는게 정석입니다.

하지만 Tree map으로 모든 데이터를 표현할 수도 있습니다.

plotly의 treemap을 이용해서 해봅시다.

In [295]:
import plotly.express as px

In [296]:
# treemap 만들기
fig = px.treemap(kospi_sector_counts,
                 path=['Sector'],
                 values='Ratio',
                 width=800,
                 height=800,
                 hover_name="Sector",
                 hover_data=['Freq', 'Ratio'],
                 custom_data=['Freq', 'Ratio'])

# 마우스를 올렸을 때, tooltip에 보이는 정보
fig.data[0].hovertemplate = \
    '<b>%{hovertext}</b><br><br>' + \
    '<b><br>Frequency: %{customdata[0]}</b>' + \
    '<br><b>Ratio: %{customdata[1]}%</b>'

fig.update_layout(
    title="The Proportion of Each Sector in KOSPI, 2020",
    title_x=0.5)
fig.write_html(img_path+"Sector_proportion.html")
fig.show()

---

# 시장에서 시가총액 비중

이 방식으로 하면 단순하게 코스피에 어떤 종목이 몇개 있는지를 알 수 있습니다.

하지만 주식시장을 볼 때는 전체 시장에서 얼마나 큰 비중을 가지고 있는지, 즉 시가총액의 비중이 어느정도 되는지를 위주로 봅니다.

시가총액 데이터를 구하고, 이를 각 섹터별로 합친 뒤, 동일한 방식으로 표현해주면 되겠습니다.

시가총액 데이터는 한국거래소(KRX)의 마켓데이터에서 불러올 수 있습니다.

하지만 감사하게도 어떤 개발자분께서 한국거래소 마켓데이터를 스크래핑 해올 수 있는 패키지를 개발해주셨습니다.

!pip install pykrx

를 실행해서 설치해주세요.

In [300]:
import pykrx as krx

pykrx 패키지에서 시가총액을 불러오는 함수는 두가지가 있습니다

1. stock.get_market_cap_by_ticker(date, market='')
2. stock.get_market_cap_by_date(fromdate, todate, ticker, frequency)

함수(1)은 해당 날짜의 모든 종목에 대해서 종목코드, 시가총액, 거래량, 거래대금, 상장주식수, 외국인보유주식수를 불러옵니다.
함수(2)는 특정 기간동안의 특정 종목의 시가총액, 거래량, 거래대금, 상장주식수를 일정한 빈도(d, m, y)에 기반하여 불러옵니다.

함수(2)를 사용해서 종목별로 데이터를 불러온 뒤, 취합하면 되겠네요.

테스트로 하나만 불러와봅시다.

불러온 다음, 시가총액 열만 취합하면 되겠네요.


In [312]:
stock_list = []
for i in range(len(kospi_list)):
    code = kospi_list['Symbol'].iloc[i]
    name = kospi_list['Name'].iloc[i]
    stock = [code, name]
    stock_list.append(stock)
    
marcap_list = [krx.stock.get_market_cap_by_date("20200319", "20201224", ticker=code, freq='d')['시가총액'] for code, name in stock_list]
marcap = pd.concat(marcap_list, axis=1)
marcap.columns = [name for code, name in stock_list]

# 불러온 데이터는 항상 저장해줍시다
marcap.to_csv(dat_path+"market_cap.csv")

In [314]:
marcap.head(5)

,AJ네트웍스,AK홀딩스,BGF,BGF리테일,BNK금융지주,BYC,CJ,CJ CGV,CJ대한통운,CJ씨푸드,...,효성티앤씨,효성화학,후성,휠라홀딩스,휴니드,휴비스,휴스틸,휴켐스,흥국화재,흥아해운
날짜,,,,,,,,,,,,,,,,,,,,,
2020-03-19,98795,158971,280450,1883946,1248332,94317,1613488,300491,2897168,73119,...,338425,162696,426917,1278834,53641,109710,32011,517114,85443,27840
2020-03-20,95517,206662,283322,2099995,1293963,94941,1686430,331175,3045448,76712,...,380403,194279,489890,1290985,57170,117300,33535,539597,97970,28304
2020-03-23,92240,196064,260350,2004933,1189664,84635,1549299,299433,3011229,80126,...,339290,179604,463960,1184668,53641,111435,32773,500763,91225,24708
2020-03-24,101136,201363,280929,2108637,1248332,89945,1747702,318478,3011229,83180,...,372613,192684,513968,1266684,58229,118680,34412,543685,105358,25636
2020-03-25,114246,233157,300072,2117278,1360780,95878,1893587,344929,2988417,86414,...,411563,225223,541750,1643348,62534,125062,36622,578432,116279,27492


데이터가 잘 취합되었네요.

이제 섹터별로 일자별 시가총액을 취합해봅시다.

순서는 다음과 같습니다.

1. 섹터별 종목 리스트를 가져오기
2. 해당하는 종목의 일자별 시가총액을 가져옵니다.
3. 데이터 프레임에 추가합니다.

In [481]:
# 데이터 프레임 만들기
sector_marcap = pd.DataFrame(kospi['Date']).reset_index(drop=True)
sector_marcap['Date'] = pd.to_datetime(sector_marcap['Date'])
sector_marcap.set_index('Date', inplace=True)

for sector in kospi_sector_list:
    companies_in_sector = list(kospi_list[kospi_list['Sector']==sector]['Name'].unique())
    sector_subset = marcap[marcap.columns.intersection(companies_in_sector)]
    sector_sum = list(sector_subset.sum(axis=1))
    sector_marcap[sector] = sector_sum
    
sector_marcap['코스피'] = sector_marcap.sum(axis=1)
marcap_growth = (sector_marcap / sector_marcap.iloc[0] - 1)*100

In [585]:
sector_mcap = dict(sector_marcap.iloc[-1])
sector_growth = dict(marcap_growth.iloc[-1])

kospi_sector['MarketCap'] = None
kospi_sector['Growth'] = None

for i in range(len(kospi_sector)):
    key = kospi_sector['Name'].loc[i]
    
    val1 = sector_growth[key]
    kospi_sector['Growth'].loc[i] = round(val1, 2)
    
    val2 = sector_mcap[key]
    kospi_sector['MarketCap'].loc[i] = val2

/home/kh/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [565]:
# treemap 만들기
fig = px.treemap(kospi_sector,
                 path=['Name'],
                 values='MarketCap',
                 width=800,
                 height=800,
                 hover_name="Name",
                 hover_data=['MarketCap', 'Growth'])

# 마우스를 올렸을 때, tooltip에 보이는 정보
fig.data[0].hovertemplate = \
    '<b>%{hovertext}</b><br><br>' + \
    '<br><b>Growth: %{customdata[1]}%</b>' + \
    '<br><b>Market Cap: %{customdata[0]}원'


fig.update_layout(
    title="The Proportion of Each Sector in KOSPI with Growth, 2020",
    title_x=0.5)
fig.write_html(img_path+"Sector_proportion.html")
fig.show()

여기까지 만들고나니 욕심이 생깁니다.

이 표 안에, 각각 종목들을 넣어줄 수는 없을까요?

할 수는 있습니다.

하지만 데이터프레임이 조금 많이 더러워진다는 단점이 있긴합니다.

공부 삼아서 그냥 해봅시다

In [503]:
kospi_sector = kospi_sector.rename(columns={'Sector':'Name'})
kospi_sector

,Name,Freq,Ratio,MarketCap,Growth
0,1차 비철금속 제조업,11,1.46,10283187,47.1862
1,1차 철강 제조업,33,4.38,34787623,106.164
2,가구 제조업,6,0.80,2339155,114.851
3,가전제품 및 정보통신장비 소매업,1,0.13,743643,158.197
4,가정용 기기 제조업,1,0.13,707446,109.553
...,...,...,...,...,...
118,항공 여객 운송업,7,0.93,11385050,131.239
119,"항공기,우주선 및 부품 제조업",2,0.27,3878886,61.0777
120,해상 운송업,4,0.53,7472318,248.416
121,화학섬유 제조업,6,0.80,3021148,126.788


In [572]:
stock_returns = (stock_prices / stock_prices.iloc[0] - 1.0 ) * 100
returns = dict(stock_returns.iloc[-1])
stock_marcap = dict(marcap.iloc[-1])

kospi_stock = pd.DataFrame([kospi_list['Name'], kospi_list['Sector']]).transpose()
kospi_stock = kospi_stock.rename(columns={'Sector':'Parent'})

kospi_stock['portion'] = None
kospi_stock['MarketCap'] = None
kospi_stock['Growth'] = None

for i in range(len(kospi_stock)):
    key = kospi_stock['Name'].iloc[i]
    parent = kospi_stock['Parent'].iloc[i]
    kospi_stock['Growth'].iloc[i] = round(returns[key], 2)
    kospi_stock['MarketCap'].iloc[i] = stock_marcap[key]
    
sector_and_stock = pd.concat([kospi_sector, kospi_stock], axis=0)

In [573]:
sector_and_stock[sector_and_stock['Parent']=='통신 및 방송 장비 제조업']

,Name,Freq,Ratio,MarketCap,Growth,portion,Parent
136,LG전자,NaN,NaN,18328555,156.59,0.03795,통신 및 방송 장비 제조업
455,대동전자,NaN,NaN,46997,45.22,0.0001,통신 및 방송 장비 제조업
510,대유플러스,NaN,NaN,90180,46.68,0.00019,통신 및 방송 장비 제조업
963,삼성전자,NaN,NaN,464449082,81.14,0.9616,통신 및 방송 장비 제조업
2587,휴니드,NaN,NaN,79332,47.89,0.00016,통신 및 방송 장비 제조업


In [588]:
# treemap 만들기
fig = px.treemap(sector_and_stock,
                 names = 'Name',
                 parents='Parent',
                 values='MarketCap',
                 width=1000,
                 height=1000,
                 hover_name="Name",
                 hover_data=['MarketCap', 'Growth'])

# 마우스를 올렸을 때, tooltip에 보이는 정보
fig.data[0].hovertemplate = \
    '<b>%{hovertext}</b><br><br>' + \
    '<br><b>Growth: %{customdata[1]}%</b>' + \
    '<br><b>Market Cap: %{customdata[0]}'


fig.update_layout(
    title="The Proportion of Each Sector in KOSPI with Growth, 2020",
    title_x=0.5)
fig.write_html(img_path+"Sector_stock_proportion.html")
fig.show()